In [1]:
from sklearn.metrics import classification_report
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
import pandas as pd

data = pd.read_csv('Petitions.csv')[:100]
data.head()

,id,public_petition_text,reason_category
0,3168490,снег на дороге,Благоустройство
1,3219678,очистить кабельный киоск от рекламы,Благоустройство
2,2963920,"Просим убрать все деревья и кустарники, которы...",Благоустройство
3,3374910,Неудовлетворительное состояние парадной - надп...,Содержание МКД
4,3336285,Граффити,Благоустройство


In [3]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
stop_words = set(stopwords.words('russian'))

def preprocess(text):
    text = text.lower()
    tokens = word_tokenize(text)
    filtered_tokens = [w for w in tokens if w.isalpha() and w not in stop_words]
    stemmer = SnowballStemmer("russian")
    stems = [stemmer.stem(w) for w in filtered_tokens]
    return ' '.join(stems)

data['preprocessed_text'] = data['public_petition_text'].apply(preprocess)

data.to_csv('preprocessed_dataset.csv', index=False)
data.head()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,id,public_petition_text,reason_category,preprocessed_text
0,3168490,снег на дороге,Благоустройство,снег дорог
1,3219678,очистить кабельный киоск от рекламы,Благоустройство,очист кабельн киоск реклам
2,2963920,"Просим убрать все деревья и кустарники, которы...",Благоустройство,прос убра дерев кустарник котор вышл предел га...
3,3374910,Неудовлетворительное состояние парадной - надп...,Содержание МКД,неудовлетворительн состоян парадн надпис двер ...
4,3336285,Граффити,Благоустройство,граффит


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()

tfidf_matrix = vectorizer.fit_transform(data['preprocessed_text'])

feature_names = vectorizer.get_feature_names_out()

print(tfidf_matrix.toarray())
print("Имена признаков:", feature_names)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Имена признаков: ['автотранспорт' 'администрац' 'адмира' 'адрес' 'актуальн' 'апрел' 'аптек'
 'арматур' 'архивн' 'асфальт' 'безопасн' 'бетон' 'благоустройств'
 'ближайш' 'блок' 'больш' 'будн' 'бумаг' 'бутылк' 'ведр' 'веревочн'
 'вернул' 'весн' 'вис' 'вкручива' 'вмеша' 'вне' 'внутрен' 'внутридворов'
 'вод' 'возл' 'возможн' 'вокруг' 'вопрос' 'восстанов' 'восстановлен'
 'вплот' 'врем' 'времен' 'всем' 'вся' 'втор' 'вход' 'входн' 'вывеск'
 'вывесок' 'вывоз' 'выкинут' 'вынужд' 'вышл' 'гаврск' 'газон' 'гараж'
 'гашек' 'ген' 'гниет' 'год' 'гололед' 'гор' 'горк' 'горяч' 'гост'
 'гражданск' 'графит' 'граффит' 'грудн' 'гряз' 'грязн' 'дал' 'дан' 'две'
 'двер' 'двор' 'дворов' 'дела' 'демонтаж' 'демонтирова' 'ден' 'дерев'
 'деревя' 'дет' 'детск' 'детьм' 'директор' 'длительн' 'дни' 'дня' 'добр'
 'довольн' 'должн' 'дом' 'домофон' 'дорог' 'дорожн' 'д

In [5]:
categories_mapping = {}

unique_values = data['reason_category'].unique().tolist()
mapping_dict = {val: idx+1 for idx, val in enumerate(unique_values)}
categories_mapping['reason_category'] = mapping_dict

data['reason_category'] = data['reason_category'].map(mapping_dict)

data, mapping_dict

(         id                               public_petition_text  \
 0   3168490                                     снег на дороге   
 1   3219678                очистить кабельный киоск от рекламы   
 2   2963920  Просим убрать все деревья и кустарники, которы...   
 3   3374910  Неудовлетворительное состояние парадной - надп...   
 4   3336285                                           Граффити   
 ..      ...                                                ...   
 95  3343110  Проезжая часть и дорога во дворе очень грязные...   
 96  3149937                                             Роллет   
 97  3154345                 мусор возле контейнерной площадкой   
 98  2988709  СПб ул Шаврова25 кор 1 . 2-ая парадная. Отсутс...   
 99  3211448  На ТСОДД длительное время размещено несанкцион...   
 
     reason_category                                  preprocessed_text  
 0                 1                                         снег дорог  
 1                 1                         о

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix, data['reason_category'], test_size=0.2, random_state=42)
y_test

,reason_category
83,1
53,2
70,1
45,1
44,2
39,1
22,2
80,1
10,1
0,1


In [7]:
import torch
import numpy as np
from torch.utils.data import TensorDataset, DataLoader

X_train_tensor = torch.tensor(X_train.todense().astype(np.float32))
y_train_tensor = torch.tensor(y_train.values)

X_test_tensor = torch.tensor(X_test.todense().astype(np.float32))
y_test_tensor = torch.tensor(y_test.values)

In [8]:
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

In [9]:
batch_size = 32

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [10]:
import torch
import numpy as np
from torch.utils.data import TensorDataset, DataLoader

X_train_tensor = torch.tensor(X_train.todense().astype(np.float32))
y_train_tensor = torch.tensor(y_train.values)

X_test_tensor = torch.tensor(X_test.todense().astype(np.float32))
y_test_tensor = torch.tensor(y_test.values)

In [ ]:
# z - gate обновления
# r - gate сброса
# h с волной - скрытое состояние
# h - окончательное скрытое состояние

class MyGRUBlock(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(MyGRUBlock, self).__init__()

        self.Wz = nn.Linear(input_size, hidden_size)
        self.Uz = nn.Linear(hidden_size, hidden_size)

        self.Wr = nn.Linear(input_size, hidden_size)
        self.Ur = nn.Linear(hidden_size, hidden_size)

        self.Wh = nn.Linear(input_size, hidden_size)
        self.Uh = nn.Linear(hidden_size, hidden_size)

    def forward(self, xt, ht_prev):
        zt = torch.sigmoid(self.Wz(xt) + self.Uz(ht_prev))

        rt = torch.sigmoid(self.Wr(xt) + self.Ur(ht_prev))

        ht_with_wave = torch.tanh(self.Wh(xt) + self.Uh(rt * ht_prev))

        ht = (1 - zt) * ht_with_wave + zt * ht_prev

        return ht

In [12]:
class MyGRUModel(nn.Module):
    def __init__(self, embed_dim, hidden_size, output_size):
        super(MyGRUModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.gru_block = MyGRUBlock(embed_dim, hidden_size)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, inputs):
      embedded = self.embedding(inputs)
      current_batch_size = inputs.size(0)
      ht_prev = torch.zeros(current_batch_size, hidden_size)
      for i in range(inputs.size(1)):
          ht_prev = self.gru_block(embedded[:,i,:], ht_prev)
      outputs = self.fc(ht_prev)
      return outputs

In [15]:
embed_dim = 16
hidden_size = 128
output_size = 16
learning_rate = 0.001
num_epochs = 5

vocab_size = len(vectorizer.get_feature_names_out())

my_model = MyGRUModel(embed_dim, hidden_size, output_size)

criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(my_model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    total_loss = 0
    for inputs, targets in train_dataloader:
        optimizer.zero_grad()
        inputs, targets = inputs.long(), targets
        # print(1)
        outputs = my_model(inputs)
        # print(2)
        loss = criterion(outputs, targets)
        # print(3)
        loss.backward()
        # print(4)
        optimizer.step()
        # print(5)
        total_loss += loss.item()
        # print(6)
    average_loss = total_loss / len(train_dataloader)
    print(f'Эпоха {epoch+1}: Средняя потеря - {average_loss:.4f}')

Эпоха 1: Средняя потеря - 2.4750
Эпоха 2: Средняя потеря - 2.0770
Эпоха 3: Средняя потеря - 1.6462
Эпоха 4: Средняя потеря - 1.4686
Эпоха 5: Средняя потеря - 1.3107


In [14]:
my_model.eval()

with torch.no_grad():
    outputs = my_model(X_test_tensor.long())
    probs = outputs.softmax(dim=-1)
    predictions = probs.argmax(dim=-1)

    y_true = y_test_tensor.cpu().numpy()
    y_pred = predictions.cpu().numpy()

print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           1       0.70      1.00      0.82        14
           2       0.00      0.00      0.00         5
           4       0.00      0.00      0.00         1

    accuracy                           0.70        20
   macro avg       0.23      0.33      0.27        20
weighted avg       0.49      0.70      0.58        20



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
